# Python ile Birliktelik Kuralları Analizi

In [71]:
# Gerekli kütüphane ve modüllerin yüklenmesi
import pandas as pd
import numpy as np
import mlxtend as ml
import random

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Veriseti Oluşturma

In [72]:
dataset = []

# Elimizde 40 adet ürün kodu var
urunler = [ 'K1U0001',
            'K1U0002',
            'K1U0003',
            'K1U0004',
            'K1U0005',
            'K1U0006',
            'K1U0007',
            'K1U0008',
            'K1U0009',
            'K1U0010',
            'K1U0011',
            'K1U0012',
            'K1U0013',
            'K1U0014',
            'K1U0015',
            'K1U0016',
            'K1U0017',
            'K1U0018',
            'K1U0019',
            'K1U0020',
            'K2U0001',
            'K2U0002',
            'K2U0003',
            'K2U0004',
            'K2U0005',
            'K2U0006',
            'K2U0007',
            'K2U0008',
            'K2U0009',
            'K2U0010',
            'K2U0011',
            'K2U0012',
            'K2U0013',
            'K2U0014',
            'K2U0015',
            'K2U0016',
            'K2U0017',
            'K2U0018',
            'K2U0019',
            'K2U0020']

# Elimizdeki ürün kodlarıyla 1000 tane rastgele gözlem kümesi oluşturuyoruz
for i in range(1, 1001):
    k = random.randrange(1, len(urunler)+1)
    dataset.append(random.choices(urunler, k=k))
    
    

In [73]:
# Oluşturulan gözlem kümelerini dataset.csv dosyasına tablo şeklinde aktarıyoruz
df = pd.DataFrame(dataset)
df.to_csv('dataset.csv', index=False)

In [74]:
# Gözlem sayısımızı kontrol ediyoruz
print(len(df))

1000


In [75]:
# Oluşturduğumuz tablodan ilk 5 gözlem değerini göstermek için
df.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,K2U0015,K2U0012,K2U0020,K2U0012,K1U0012,K2U0005,K1U0010,K1U0015,K2U0017,K2U0019,...,None,None,None,None,None,None,None,None,None,None
1,K1U0004,K2U0011,K2U0002,K2U0007,K2U0002,K2U0016,K2U0009,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,K1U0010,K2U0014,K2U0005,K1U0016,K1U0010,K2U0004,K2U0013,K1U0013,K2U0016,K2U0012,...,K1U0004,None,None,None,None,None,None,None,None,None
3,K2U0017,K1U0005,K1U0017,K1U0017,K1U0018,K2U0012,K1U0007,K2U0007,K2U0002,K2U0007,...,None,None,None,None,None,None,None,None,None,None
4,K1U0003,K1U0016,K2U0008,K1U0005,K2U0017,K1U0003,K1U0009,K2U0011,K2U0019,K1U0004,...,K1U0010,K2U0002,K1U0016,K1U0018,K2U0004,K1U0018,K1U0017,K1U0006,None,None


In [76]:
# Oluşturduğumuz tablodan son 5 gözlem değerini göstermek için
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
995,K2U0014,K2U0010,K2U0009,K1U0017,K1U0005,K1U0019,K1U0006,K2U0016,K2U0018,K2U0020,...,None,None,None,None,None,None,None,None,None,None
996,K1U0016,K1U0010,K1U0012,K1U0004,K2U0001,K2U0001,K1U0004,K2U0014,K1U0017,K2U0014,...,K1U0009,K1U0001,K1U0001,K1U0006,K2U0012,K2U0001,K1U0001,K2U0008,None,None
997,K2U0001,K2U0003,K2U0008,K1U0017,K2U0020,K2U0001,K2U0008,K2U0001,K1U0009,K1U0007,...,None,None,None,None,None,None,None,None,None,None
998,K1U0012,K1U0011,K1U0004,K1U0010,K2U0018,K2U0010,K1U0004,K2U0011,K2U0013,K1U0011,...,None,None,None,None,None,None,None,None,None,None
999,K1U0018,K2U0002,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [77]:
# Veri setini oluşturduktan sonra nested list (iç içe liste) tipindeki verilerimizi tablosal bir yapıya çevirmemiz gerekiyor
# Bunun için mlxtend modülü içerisinde yer alan preprocessing sınıfı içerisinde TransactionEncoder fonksiyonunu kullanacağız

t = TransactionEncoder()
td = t.fit(dataset).transform(dataset)
df = pd.DataFrame(td, columns=t.columns_)

In [78]:
# Tabloya çevirdiğimiz verisetimizin ilk beş değerini tekrar kontrol ediyoruz
df.head()

,K1U0001,K1U0002,K1U0003,K1U0004,K1U0005,K1U0006,K1U0007,K1U0008,K1U0009,K1U0010,...,K2U0011,K2U0012,K2U0013,K2U0014,K2U0015,K2U0016,K2U0017,K2U0018,K2U0019,K2U0020
0,True,True,False,False,False,False,False,False,False,True,...,False,True,False,False,True,False,True,True,True,True
1,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2,True,True,False,True,True,False,False,False,True,True,...,False,True,True,True,False,True,True,True,True,True
3,False,False,False,False,True,False,True,False,False,False,...,True,True,False,False,False,True,True,False,False,False
4,True,False,True,True,True,True,False,False,True,True,...,True,True,False,False,False,False,True,True,True,True


## Model Oluşturma

Modeli oluşturmak için APRIORI algoritmasını kullanacağız

In [79]:
# En az %20 destek şartımızla algoritmayı çalıştırıyoruz
apriori(df, min_support=0.2)

,support,itemsets
0,0.378,(0)
1,0.371,(1)
2,0.395,(2)
3,0.354,(3)
4,0.384,(4)
5,0.392,(5)
6,0.388,(6)
7,0.384,(7)
8,0.391,(8)
9,0.358,(9)


In [80]:
# Yukarıdaki tabloda "itemset" kolunundaki numaralar yerin ürün kodlarını göstermek için "use_colnames=True" kod parçasını ekliyoruz
apriori(df, min_support=0.20, use_colnames=True)

,support,itemsets
0,0.378,(K1U0001)
1,0.371,(K1U0002)
2,0.395,(K1U0003)
3,0.354,(K1U0004)
4,0.384,(K1U0005)
5,0.392,(K1U0006)
6,0.388,(K1U0007)
7,0.384,(K1U0008)
8,0.391,(K1U0009)
9,0.358,(K1U0010)


In [81]:
# Kaç tane gözlemin belirlediğimiz %20lik destek kuralımıza uyguduğunu kontrol etmek için 
print(len(apriori(df, min_support=0.20)))

56


In [82]:
# Gözlem değerlerini atama 
gozlem_kumesi = apriori(df, min_support=0.20, use_colnames=True)

# Gözlem kümemizde her bir gözlemin kaç değer içerdiğini içeren bir kolon eklemek için
gozlem_kumesi['length'] = gozlem_kumesi['itemsets'].apply(lambda x: len(x))
gozlem_kumesi

,support,itemsets,length
0,0.378,(K1U0001),1
1,0.371,(K1U0002),1
2,0.395,(K1U0003),1
3,0.354,(K1U0004),1
4,0.384,(K1U0005),1
5,0.392,(K1U0006),1
6,0.388,(K1U0007),1
7,0.384,(K1U0008),1
8,0.391,(K1U0009),1
9,0.358,(K1U0010),1


In [83]:
# Şimdi istediğimiz kriterlere uyan gözlemi daha rahat bir şekilde bulabiliriz
# Örnek: İçinde 2 ürün olan ve destek değeri %20 olan gözlem kümesi için

gozlem_kumesi[ (gozlem_kumesi['length'] == 2) & (gozlem_kumesi['support'] >= 0.2) ]

,support,itemsets,length
40,0.201,"(K1U0001, K1U0007)",2
41,0.201,"(K1U0019, K1U0006)",2
42,0.205,"(K2U0009, K1U0011)",2
43,0.203,"(K1U0015, K1U0013)",2
44,0.206,"(K1U0013, K1U0018)",2
45,0.203,"(K1U0019, K1U0013)",2
46,0.203,"(K1U0015, K1U0018)",2
47,0.202,"(K1U0019, K1U0015)",2
48,0.214,"(K2U0009, K1U0015)",2
49,0.201,"(K2U0010, K1U0018)",2


In [85]:
# Veya ürünler bazında filtreleme yapabiliriz
gozlem_kumesi[ gozlem_kumesi['itemsets'] == {'K1U0001', 'K1U0007'} ]

,support,itemsets,length
40,0.201,"(K1U0001, K1U0007)",2


#### İlgilendiğimiz metriğe göre (confidence, lift, conviction ve vd.) Association Rules tablosunu oluşturmak için

In [86]:
# Burada metrik olarak Confidence ve değerini 0.3 (%30) seçtik.
rules1 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.30)

In [87]:
print("Oluşan Kural Sayısı:", len(rules1))

Oluşan Kural Sayısı: 1478


In [88]:
# Confidence değeri alçalan şekilde tabloda sıralamak için
rules1 = rules1.sort_values(['confidence'], ascending=False)
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
66,(K1U0001),(K2U0019),0.358358,0.409409,0.193193,0.539106,1.316790,0.046478,1.281403
912,(K1U0015),(K2U0019),0.352352,0.409409,0.187187,0.531250,1.297601,0.042931,1.259927
185,(K2U0006),(K1U0003),0.399399,0.416416,0.209209,0.523810,1.257898,0.042893,1.225526
1188,(K2U0003),(K2U0008),0.363363,0.409409,0.190190,0.523416,1.278466,0.041426,1.239216
894,(K1U0015),(K2U0008),0.352352,0.409409,0.184184,0.522727,1.276784,0.039928,1.237428
...,...,...,...,...,...,...,...,...,...
1473,(K2U0019),(K2U0018),0.409409,0.361361,0.160160,0.391198,1.082567,0.012215,1.049009
23,(K1U0014),(K1U0001),0.384384,0.358358,0.150150,0.390625,1.090040,0.012403,1.052950
176,(K1U0003),(K2U0002),0.416416,0.352352,0.162162,0.389423,1.105209,0.015437,1.060714
3,(K1U0003),(K1U0001),0.416416,0.358358,0.162162,0.389423,1.086686,0.012936,1.050878


#### Yorum: ID bilgisi 66 olan satırı inceleyecek olursak;
